In [86]:
import pulp
import matplotlib_inline as plt

In [87]:
prob = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)
prob2 = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)
prob3 = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

In [88]:
import numpy as np

def generate_random_directed_graph(num_nodes, num_edges, cost_range=(1, 10)):
    # check the number of nodes and edges
    if num_edges < num_nodes or num_edges > num_nodes * (num_nodes - 1) // 2:
        raise ValueError("Number of edges must be at least num_nodes and no more than num_nodes * (num_nodes - 1) // 2.")

    # product nodes include the r
    nodes = [chr(i) for i in range(97, 97 + num_nodes)]

    # create a cycle to ensure the strong connection
    cycle_edges = [(nodes[i], nodes[(i + 1) % num_nodes]) for i in range(num_nodes)]
    
    # produce other possible edge
    possible_edges = [(nodes[i], nodes[j]) for i in range(num_nodes) for j in range(num_nodes) if i != j and (nodes[j], nodes[i]) not in cycle_edges]

    # remove the duplicated edge
    np.random.shuffle(possible_edges)
    remaining_edges_count = num_edges - len(cycle_edges)
    selected_edges = cycle_edges + possible_edges[:remaining_edges_count]
    
    # produce edges with costs
    edges_with_costs = {edge: np.random.randint(cost_range[0], cost_range[1] + 1) for edge in selected_edges}

    return nodes, edges_with_costs

# 使用该函数生成一个随机有向图
num_nodes = 10# 节点数量
num_edges = 45# 总边数，至少等于节点数
nodes, edges_with_costs = generate_random_directed_graph(num_nodes, num_edges)
#for keys in edges_with_costs:
    #edges_with_costs[keys] = float(edges_with_costs[keys]/100)

print("Nodes:", nodes)
print("Directed edges with costs:", edges_with_costs)

Nodes: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
Directed edges with costs: {('a', 'b'): 1, ('b', 'c'): 5, ('c', 'd'): 6, ('d', 'e'): 4, ('e', 'f'): 1, ('f', 'g'): 2, ('g', 'h'): 7, ('h', 'i'): 5, ('i', 'j'): 6, ('j', 'a'): 8, ('d', 'b'): 1, ('j', 'f'): 7, ('e', 'g'): 1, ('j', 'c'): 1, ('a', 'd'): 2, ('i', 'e'): 5, ('j', 'd'): 7, ('c', 'a'): 1, ('f', 'a'): 5, ('b', 'h'): 6, ('g', 'b'): 3, ('b', 'g'): 3, ('f', 'c'): 1, ('f', 'b'): 10, ('e', 'j'): 8, ('e', 'c'): 2, ('b', 'f'): 6, ('h', 'f'): 4, ('j', 'e'): 8, ('h', 'd'): 9, ('d', 'g'): 9, ('h', 'e'): 9, ('i', 'b'): 3, ('c', 'f'): 7, ('a', 'e'): 6, ('j', 'g'): 9, ('h', 'c'): 7, ('g', 'e'): 7, ('g', 'd'): 8}


In [89]:
sp = input('Please select the root: ')
sp

'a'

In [90]:
import pulp
x = pulp.LpVariable.dicts('x',(nodes,nodes),0,1,cat=pulp.LpInteger)
y = pulp.LpVariable.dicts('y',(nodes,nodes),0,1,cat=pulp.LpInteger)
z = pulp.LpVariable.dicts('z',(nodes,nodes),0,1,cat=pulp.LpInteger)

In [91]:
prob +=pulp.lpSum(edges_with_costs[u,v] * x[u][v] for u, v in edges_with_costs)
prob2 +=pulp.lpSum(edges_with_costs[u,v] * y[u][v] for u, v in edges_with_costs)
prob3 +=pulp.lpSum(edges_with_costs[u,v] * z[u][v] for u, v in edges_with_costs)

In [92]:
for v in nodes:
    if v != sp:
        prob +=pulp.lpSum(x[u][v] for u in nodes if (u,v) in edges_with_costs) == 1
    prob3 +=pulp.lpSum(z[u][v] for u in nodes if (u,v) in edges_with_costs) >= 1

for u in nodes:
    if u != sp:
        prob2 +=pulp.lpSum(y[u][v] for v in nodes if (u,v) in edges_with_costs) == 1
    prob3 +=pulp.lpSum(z[u][v] for v in nodes if (u,v) in edges_with_costs) >= 1

In [93]:
prob += pulp.lpSum(x[u][sp] for u in nodes if (u,sp) in edges_with_costs) == 0
prob += pulp.lpSum(x[sp][v] for v in nodes if (sp,v) in edges_with_costs)>= 1

prob2 += pulp.lpSum(y[u][sp] for u in nodes if (u,sp) in edges_with_costs) >=1
prob2 += pulp.lpSum(y[sp][v] for v in nodes if (sp,v) in edges_with_costs) ==0

In [94]:
from pulp import *
n = len(nodes)
subsets = [tuple(c) for c in allcombinations(nodes,n-1)]
subset = []
for s in subsets:
    if sp in s:
        subset.append(s)

In [95]:
for s in subset:
    sc = list(set(nodes)-set(s)) # complement of subset s
    summation = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation += x[u][v]
    prob += summation >= 1
    summation2 = 0.0
    for u in sc:
        for v in s:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation2 += y[u][v]
    prob2 += summation2 >= 1
    
for s in subsets:
    sc = list(set(nodes) - set(s))  # complement of subset s
    summation3 = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs:  # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation3 += z[u][v]
    prob3 += summation3 >= 1  

In [96]:
prob.solve()
print ("Status:", LpStatus[prob.status])
prob2.solve()
print ("Status:", LpStatus[prob2.status])
prob3.solve()
print ("Status:", LpStatus[prob3.status])

Status: Optimal
Status: Optimal
Status: Optimal


In [104]:
print ("Optimal Solution for out")
resultout = []
for u in nodes:
	for v in nodes:
		if x[u][v].value() == 1:
			resultout.append((u,v))
resultout

Optimal Solution for out


[('a', 'b'),
 ('a', 'd'),
 ('b', 'h'),
 ('d', 'e'),
 ('e', 'f'),
 ('e', 'g'),
 ('h', 'i'),
 ('i', 'j'),
 ('j', 'c')]

In [108]:
costout = 0.0
for e in resultout:
    costout+= edges_with_costs[e]
costout

27.0

In [ ]:
print ("Optimal Solution for in")
resultin = []
for u in nodes:
	for v in nodes:
		if y[u][v].value() == 1:
		    resultin.append((u,v))
resultin

In [110]:
costin = 0.0
for e in resultin:
    costin+= edges_with_costs[e]
costin

20.0

In [111]:
print ("Optimal Solution for strongly connected subgraph")
resultall = []
for u in nodes:
	for v in nodes:
		if z[u][v].value() == 1:
			resultall.append((u,v))
resultall

Optimal Solution for strongly connected subgraph


[('a', 'd'),
 ('b', 'h'),
 ('c', 'a'),
 ('d', 'e'),
 ('e', 'f'),
 ('e', 'g'),
 ('f', 'c'),
 ('g', 'b'),
 ('h', 'i'),
 ('i', 'j'),
 ('j', 'c')]

In [113]:
costall = 0.0
for e in resultall:
    costall+= edges_with_costs[e]
costall

31.0

In [114]:
print ("Combine solution for out and in")
result=[]
for u in nodes:
	for v in nodes:
		if x[u][v].value() == 1:
			result.append((u,v))
		elif y[u][v].value() == 1:
			result.append((u,v))
            
result

Combine solution for out and in


[('a', 'b'),
 ('a', 'd'),
 ('b', 'c'),
 ('b', 'h'),
 ('c', 'a'),
 ('d', 'b'),
 ('d', 'e'),
 ('e', 'f'),
 ('e', 'g'),
 ('f', 'c'),
 ('g', 'b'),
 ('h', 'f'),
 ('h', 'i'),
 ('i', 'b'),
 ('i', 'j'),
 ('j', 'c')]

In [115]:
costcom = 0.0
for e in result:
    costcom+= edges_with_costs[e]
costcom

20.0

In [116]:
print("Compare solution combine of out/in and for strongly connected subgraph")
print("result for strongly connected subgraph",resultall)
print("result for combine of out/in:", result)
print("Two result are same:", resultall==result)

Compare solution combine of out/in and for strongly connected subgraph
result for strongly connected subgraph [('a', 'd'), ('b', 'h'), ('c', 'a'), ('d', 'e'), ('e', 'f'), ('e', 'g'), ('f', 'c'), ('g', 'b'), ('h', 'i'), ('i', 'j'), ('j', 'c')]
result for combine of out/in: [('a', 'b'), ('a', 'd'), ('b', 'c'), ('b', 'h'), ('c', 'a'), ('d', 'b'), ('d', 'e'), ('e', 'f'), ('e', 'g'), ('f', 'c'), ('g', 'b'), ('h', 'f'), ('h', 'i'), ('i', 'b'), ('i', 'j'), ('j', 'c')]
Two result are same: False
